## Imports 
Para realização do projeto foram escolhidas as seguintes bibliotecas, juntamente com elas estarão listadas as principais funcionalidades das mesmas nesse contexto:  
* Pandas - Processamento de dados e leitura do arquivo .csv
* Numpy - Operações envolvendo álgebra linear
* Matplotlib - Visualização de dados
* Nltk - Processamento de linguagem natural
* Sklearn - Mineração de dados e análise de dados
* Seaborn - Visualização de dados

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

df_train = pd.read_csv('train.csv')

Analisando o cabeçalho do _data frame_

In [3]:
df_train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


Verificando a quantidade de tweets marcados como discurso de ódio ou não para decisão de métrica de avaliação.

In [6]:
print(str(len(df_train[df_train.label == 1])) + ' Hatred Tweets')
print(str(len(df_train[df_train.label == 0])) + ' Non-Hatred Tweets')

2242 Hatred Tweets
29720 Non-Hatred Tweets


Baseado no OutPut obtido da quantidade de tweets que contém discurso de ódio ou não, pode-se afimar que a métrica F1 score será a melhor para esse contexto, por

## Pre-processamento dos dados
O dataset apresenta uma série de menções a usuários e para não expor nenhum, os nomes foram substituídos por _"user"_. Logo, esse tipo de infomação deve ser descartada por não apresentar nenhum valor para a análise final.  
Já as hashtags presentes nos tweets serão movidas para outra coluna para futuro tratamento.